In [1]:
from DcwAggregation import *

verbose = True
extraVerbose = False
if verbose:
    import pprint
    # from IPython.core.display import display
    pprinter = pprint.PrettyPrinter(indent=4)

# get_ipython().magic('matplotlib inline')

# ## Globally relevant variables

saveIdentifiedLineDetails = True  # Need to do this to recompute for every codebook
applyDoubleLineFix = False
applyDoubleWordFilter = True
applyDoubleLineFilter = True
classificationBaseDirectory = 'C:/Users/danny/Repos/text_segmentation/consensus/classifications/'
consensusBaseDirectory = 'C:/Users/danny/Repos/text_segmentation/consensus/consensus/'
databaseNamePattern = 'dcwConsensus_{mss_label}'
aggregatedDataFileNamePattern = 'decoding-the-civil-war-aggregated_{mss_label}.txt'
# 'decoding-the-civil-war-consensus-linewise.csv'
aggregatedDataCsvFileNamePattern = 'decoding-the-civil-war-consensus-linewise_{mss_label}.csv'
# 'decoding-the-civil-war-consensus-subjectwise.csv'
aggregatedDataSubjectWiseCsvFileNamePattern = 'decoding-the-civil-war-consensus-subjectwise_{mss_label}_withBreaks.csv'
# 'dataWithLineIDs_subset.pkl'
identifiedLineFilePathPattern = 'dataWithLineIDs_subset_{mss_label}.pkl'
liveDate = dateutil.parser.parse("2016-06-20T00:00:00.00Z")

# Processing multiple classificaton files
classificationCsvFiles = glob.glob(
    '{}/*.csv'.format(classificationBaseDirectory))

In [2]:
len(classificationCsvFiles)

49

In [3]:
# ledgerIndex = 30
for sampleDataFileName in classificationCsvFiles:
    print('Processing {}...'.format(sampleDataFileName))
    
    try:
        mssLabel = sampleDataFileName.split('/')[-1][len(
            'classification_export_'):-4]
        aggregatedDataFileName = aggregatedDataFileNamePattern.format(
            mss_label=mssLabel)  # 'decoding-the-civil-war-aggregated.txt'
        aggregatedDataCsvFileName = aggregatedDataCsvFileNamePattern.format(
            mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-linewise.csv'
        aggregatedDataSubjectWiseCsvFileName = aggregatedDataSubjectWiseCsvFileNamePattern.format(
            mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-subjectwise.csv'
        identifiedLineFilePath = identifiedLineFilePathPattern.format(
            mss_label=mssLabel)  # 'dataWithLineIDs_subset.pkl'

        subjectDataFileName = 'decoding-the-civil-war-subjects-9-29-17.csv'

        subjectsFrame = loadSubjectData(subjectDataFileName)

        telegrams, nTelegramsParsed = loadTelegrams(sampleDataFileName)
        print('Parsed {} telegrams and stored {}.'.format(
            nTelegramsParsed, len(telegrams)))

        transcriptionLineStats, transcriptionLineDetailsFrame = processLoadedTelegrams(
            telegrams)

        transcriptionLineDetailsFrame = groupTranscriptionsLinewise(
            transcriptionLineDetailsFrame, 40, identifiedLineFilePath, saveIdentifiedLineDetails)

        # This is an intentional no-op
        transcriptionLineDetailsFrame = doubleLineFix(
            transcriptionLineDetailsFrame, applyDoubleLineFix=False)

        # The previous step is time consuming so serialize the processed data at
        # this stage
        if saveIdentifiedLineDetails:
            transcriptionLineDetailsFrame.to_pickle(consensusBaseDirectory + identifiedLineFilePath)

        lineGroupedTranscriptionLineDetails = processSentences(
            transcriptionLineDetailsFrame, subjectsFrame)
        saveAggregatedData(lineGroupedTranscriptionLineDetails,
                           consensusBaseDirectory + aggregatedDataCsvFileName, 
                           consensusBaseDirectory + aggregatedDataSubjectWiseCsvFileName,
                           applyDoubleWordFilter, applyDoubleLineFilter, extraVerbose)
    except:
        print("IT DIDN'T WORK FOR SOME REASON")

Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_02_05_25_17.csv...
Parsed 2177 telegrams and stored 232.
Processed 100 subjects...
Processed 200 subjects...
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_03_1_19_17.csv...
Parsed 562 telegrams and stored 63.
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_04_05_25_17.csv...
Parsed 2523 telegrams and stored 271.
Processed 100 subjects...
Processed 200 subjects...
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_05_04_20_17.csv...
IT DIDN'T WORK FOR SOME REASON
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_07_04_20_17.csv...
IT DIDN'T WORK FOR SOME REASON
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_08_0

# Testing double line

In [3]:
# ledgerIndex = 30
# for sampleDataFileName in classificationCsvFiles:
sampleDataFileName = classificationCsvFiles[0]
print('Processing {}...'.format(sampleDataFileName))

# try:
mssLabel = sampleDataFileName.split('/')[-1][len(
    'classification_export_'):-4]
aggregatedDataFileName = aggregatedDataFileNamePattern.format(
    mss_label=mssLabel)  # 'decoding-the-civil-war-aggregated.txt'
aggregatedDataCsvFileName = aggregatedDataCsvFileNamePattern.format(
    mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-linewise.csv'
aggregatedDataSubjectWiseCsvFileName = aggregatedDataSubjectWiseCsvFileNamePattern.format(
    mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-subjectwise.csv'
identifiedLineFilePath = identifiedLineFilePathPattern.format(
    mss_label=mssLabel)  # 'dataWithLineIDs_subset.pkl'

subjectDataFileName = 'decoding-the-civil-war-subjects-9-29-17.csv'

subjectsFrame = loadSubjectData(subjectDataFileName)

telegrams, nTelegramsParsed = loadTelegrams(sampleDataFileName)
print('Parsed {} telegrams and stored {}.'.format(
    nTelegramsParsed, len(telegrams)))

Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_02_05_25_17.csv...
Parsed 2177 telegrams and stored 232.


In [4]:
transcriptionLineStats, transcriptionLineDetailsFrame = processLoadedTelegrams(
    telegrams)

In [6]:
transcriptionLineDetailsFrame = groupTranscriptionsLinewise(
    transcriptionLineDetailsFrame, 40, identifiedLineFilePath, saveIdentifiedLineDetails)
transcriptionLineDetailsFrame_old = transcriptionLineDetailsFrame

Processed 100 subjects...
Processed 200 subjects...


#### Bold and old
Trying double line step by step

In [7]:
transcriptionLineDetailsFirstPass = transcriptionLineDetailsFrame.set_index(
    'bestLineIndex', drop=False, append=True)
transcriptionLineDetailsFirstPass[
    'oldBestLineIndex'] = transcriptionLineDetailsFirstPass[
        'bestLineIndex'].astype(np.int64)
transcriptionLineDetailsFirstPass = transcriptionLineDetailsFirstPass.groupby(
    level=[0, 5]).aggregate({
        'y1': np.mean,
        'y2': np.mean,
        'bestLineIndex': 'first',
        'oldBestLineIndex': 'first'
    })
transcriptionLineDetailsFirstPass['meanY'] = 0.5 * (
    transcriptionLineDetailsFirstPass['y1'] +
    transcriptionLineDetailsFirstPass['y2'])
transcriptionLineDetailsFirstPass.set_index(
    'meanY', drop=False, append=True)

y1           y2  bestLineIndex  \
subjectKey bestLineIndex meanY                                                  
1959490    0             160.254728    161.426952   159.082504              0   
           1             181.369571    182.594495   180.144646              1   
           2             224.986473    217.476181   232.496765              2   
           3             250.839420    238.323036   263.355804              3   
           4             245.775490    247.079455   244.471525              4   
           5             300.583160    299.217509   301.948811              5   
           6             357.553100    356.781583   358.324617              6   
           7             405.856900    405.331337   406.382463              7   
           8             460.178168    461.855491   458.500845              8   
           9             514.616470    516.381311   512.851629              9   
           10            566.287539    566.333662   566.241416             10   
           11            608.712910    609.738084   607.687736             11   
           12            638.764801    640.417419   637.112183             12   
           13            671.474785    671.591962   671.357607             13   
           14            719.157258    719.341831   718.972684             14   
           15            771.042826    771.431164   770.654487             15   
           16            820.425987    823.507460   817.344513             16   
           17            871.562556    873.402316   869.722797             17   
           18            920.525689    923.382454   917.668923             18   
           19            970.693299    971.827000   969.559599             19   
           20            1019.454502  1020.176998  1018.732006             20   
           21            1076.861833  1076.511887  1077.211780             21   
           22            1130.223004  1131.205259  1129.240750             22   
           23            1182.365828  1185.424682  1179.306974             23   
           24            1232.688565  1234.302006  1231.075124             24   
           25            1279.956060  1279.341193  1280.570927             25   
           26            1335.042548  1335.767206  1334.317889             26   
           27            1388.307705  1388.668820  1387.946590             27   
           28            1442.169002  1444.645951  1439.692054             28   
           29            1499.369365  1501.114228  1497.624503             29   
...                                           ...          ...            ...   
1959718    13            807.884286    809.566483   806.202089             13   
           14            857.261131    858.734147   855.788115             14   
           15            906.558593    907.835532   905.281654             15   
           16            956.713972    959.718349   953.709595             16   
           17            1009.920873  1011.819022  1008.022724             17   
           18            1062.168592  1064.513045  1059.824140             18   
           19            1111.745934  1115.776036  1107.715832             19   
           20            1165.306593  1168.945663  1161.667522             20   
           21            1922.580668  1921.945387  1923.215949             21   
1959719    0             112.620141    113.519906   111.720376              0   
           1             170.383753    169.078341   171.689165              1   
           2             238.604013    238.059872   239.148154              2   
           3             293.408789    292.823421   293.994158              3   
           4             348.283135    347.962616   348.603654              4   
           5             398.234282    398.666130   397.802433              5   
           6             444.322801    443.232990   445.412612              6   
           7             500.501535    500.964466   500.038604              7 

In [8]:
oldColumnNames = transcriptionLineDetailsFirstPass.columns
newColumnNames = [(name if name != 'bestLineIndex' else 'newBestLineIndex')
                  for name in oldColumnNames]
transcriptionLineDetailsFirstPass.columns = newColumnNames

In [9]:
thisRow = transcriptionLineDetailsFirstPass.iloc[0]
# display(thisRow)

meanYThreshold = 20
bestLineIndexDecrement = 0

for index, nextRow in transcriptionLineDetailsFirstPass.iterrows():
    if nextRow['newBestLineIndex'] == 0:  # new subject
        bestLineIndexDecrement = 0  # so reset the decrement
    elif np.abs(thisRow['meanY'] - nextRow['meanY']) < meanYThreshold:
        bestLineIndexDecrement += 1

    transcriptionLineDetailsFirstPass.loc[
        index, 'newBestLineIndex'] -= int(bestLineIndexDecrement)

    thisRow = nextRow

In [12]:
transcriptionLineDetailsFirstPass.reset_index(
    level='bestLineIndex', drop=True, inplace=True)
transcriptionLineDetailsFirstPass.columns = oldColumnNames

In [14]:
transcriptionLineDetailsSecondPass = transcriptionLineDetailsFirstPass.set_index(
    'bestLineIndex', drop=False, append=True)
transcriptionLineDetailsMismatches = transcriptionLineDetailsSecondPass[(
    transcriptionLineDetailsSecondPass['bestLineIndex'] !=
    transcriptionLineDetailsSecondPass['oldBestLineIndex'])]
transcriptionLineDetailsMismatches.reset_index(
    level='bestLineIndex', drop=True, inplace=True)
# display(transcriptionLineDetailsMismatches)
transcriptionLineDetailsMismatches

,y1,y2,bestLineIndex,oldBestLineIndex,meanY
subjectKey,,,,,
1959490,247.079455,244.471525,3.0,4,245.775490
1959490,299.217509,301.948811,4.0,5,300.583160
1959490,356.781583,358.324617,5.0,6,357.553100
1959490,405.331337,406.382463,6.0,7,405.856900
1959490,461.855491,458.500845,7.0,8,460.178168
1959490,516.381311,512.851629,8.0,9,514.616470
1959490,566.333662,566.241416,9.0,10,566.287539
1959490,609.738084,607.687736,10.0,11,608.712910
1959490,640.417419,637.112183,11.0,12,638.764801


In [15]:
for mismatchIndex, mismatchRow in transcriptionLineDetailsMismatches.iterrows():
    for index, row in transcriptionLineDetailsFrame.loc[
            mismatchIndex].iterrows():
        if row['bestLineIndex'] == mismatchRow['oldBestLineIndex']:
            transcriptionLineDetailsFrame.loc[
                mismatchIndex, 'bestLineIndex'] = mismatchRow[
                    'bestLineIndex']

#### Bold and old
back to normal algorithm

In [ ]:
# This is an intentional no-op
transcriptionLineDetailsFrame = doubleLineFix(
    transcriptionLineDetailsFrame, applyDoubleLineFix=False)

In [3]:
# The previous step is time consuming so serialize the processed data at
# this stage
if saveIdentifiedLineDetails:
    transcriptionLineDetailsFrame.to_pickle(consensusBaseDirectory + identifiedLineFilePath)

lineGroupedTranscriptionLineDetails = processSentences(
    transcriptionLineDetailsFrame, subjectsFrame)
saveAggregatedData(lineGroupedTranscriptionLineDetails,
                   consensusBaseDirectory + aggregatedDataCsvFileName, 
                   consensusBaseDirectory + aggregatedDataSubjectWiseCsvFileName,
                   applyDoubleWordFilter, applyDoubleLineFilter, extraVerbose)
# except:
# print("IT DIDN'T WORK FOR SOME REASON")

Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_02_05_25_17.csv...
Parsed 2177 telegrams and stored 232.
Processed 100 subjects...
Processed 200 subjects...
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_03_1_19_17.csv...
Parsed 562 telegrams and stored 63.
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_04_05_25_17.csv...
Parsed 2523 telegrams and stored 271.
Processed 100 subjects...
Processed 200 subjects...
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_05_04_20_17.csv...
IT DIDN'T WORK FOR SOME REASON
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_07_04_20_17.csv...
IT DIDN'T WORK FOR SOME REASON
Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_08_0